In [39]:
import pandas as pd
import numpy as np

In [40]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from Utils.data_get_utils import get_duka_data_df
from Utils.data_process_utils import slice_df_by_timerange
from datetime import timedelta
from datetime import datetime
import matplotlib.dates as mdates

BIG_TABLE_DATA_PATH="../data/yht_eur_big_table.csv"
INPUT_BAR_DATA_PATHS=["../data/EURUSD5_duka_2020.csv", "../data/EURUSD5_duka_2021.csv"]
SYMBOL="EURUSD"
OUTPUT_PATH="../data/yht_duka_5m_eur_with_floatingprofits_table.csv"

from Utils.data_get_utils import get_duka_data_df_by_list
duka_5m_df = get_duka_data_df_by_list(INPUT_BAR_DATA_PATHS, SYMBOL)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
duka_5m_df.head()

,symbol,datetime,open,high,low,close,volume
264,EURUSD,2020-01-01 22:00:00,1.12136,1.12139,1.12120,1.12125,47.60
265,EURUSD,2020-01-01 22:05:00,1.12125,1.12127,1.12125,1.12127,91.79
266,EURUSD,2020-01-01 22:10:00,1.12127,1.12127,1.12123,1.12127,64.98
267,EURUSD,2020-01-01 22:15:00,1.12127,1.12154,1.12127,1.12146,85.07
268,EURUSD,2020-01-01 22:20:00,1.12144,1.12153,1.12137,1.12146,123.93


In [42]:
# 价格序列
# 加仓初始手数，加仓间隔，加仓倍数
# 成本位，当前价格距离成本位的间距

# 部分平仓，加仓，

In [43]:
lots_factor = [2,3,4,5,8,11,15,21,29,30,30,30,30,30]

# lots_factor =        [1,1 ,2 ,2 ,3 ,3 ,3 ,4 ,4 ,6 ,6 , 21, 21, 21]
# target_profit_pips = [4,10,10,10,10,]

step_pips = 8
init_lots = 0.04
profit_per_lot_per_pip = 10
target_profit_pips = 4

price_init_in = 1.12144

ROUND_NUM = 4
LOTS_NORM = 2
DIGITS = 0.0001

prev_total_lots = init_lots 
prev_avg_in = price_init_in
prev_price_in = price_init_in

already_in = [[prev_avg_in, price_init_in, 0, 0, 0, init_lots, 0, profit_per_lot_per_pip * init_lots * target_profit_pips, 1]]

for factor in lots_factor:
    cur_lots = round(init_lots * factor, ROUND_NUM)
    cur_price_in = round(prev_price_in - step_pips * DIGITS, ROUND_NUM)
    cur_total_lots = round(prev_total_lots + cur_lots, LOTS_NORM)
    cur_avg_in = (prev_avg_in * prev_total_lots + cur_lots * cur_price_in) / (prev_total_lots + cur_lots)
    cur_avg_in = round(cur_avg_in, ROUND_NUM)
    cur_pips_loss = round(abs(price_init_in - cur_price_in)/DIGITS, ROUND_NUM)
    cur_pips_retrace = round(abs(cur_avg_in - cur_price_in)/DIGITS, ROUND_NUM)
    cur_ratio = round(cur_pips_retrace / cur_pips_loss, ROUND_NUM)
    cur_float_loss = -round(cur_total_lots * cur_pips_loss * profit_per_lot_per_pip, ROUND_NUM)
    cur_target_profit = target_profit_pips * cur_total_lots * profit_per_lot_per_pip
    cur_t_to_l = round(cur_target_profit / -cur_float_loss, 2)
    
    already_in.append([cur_avg_in, cur_price_in, 
                       cur_pips_loss, cur_pips_retrace, cur_ratio, 
                       cur_total_lots, cur_float_loss, cur_target_profit, cur_t_to_l])
    prev_total_lots = cur_total_lots
    prev_avg_in = cur_avg_in
    prev_price_in = cur_price_in

In [44]:
print("总加仓层数: ", len(lots_factor))

pd.DataFrame(already_in, columns=["初始入场位", "当前入场位", "被套点数", "回调平本(点)",
                                 "回调平本(比)", "总仓位", "总浮亏", "目标盈利", "盈亏比"])

总加仓层数:  14


,初始入场位,当前入场位,被套点数,回调平本(点),回调平本(比),总仓位,总浮亏,目标盈利,盈亏比
0,1.12144,1.12144,0.0,0.0,0.0000,0.04,0.00,1.6,1.00
1,1.12090,1.12060,8.4,3.0,0.3571,0.12,-10.08,4.8,0.48
2,1.12040,1.11980,16.4,6.0,0.3659,0.24,-39.36,9.6,0.24
3,1.11980,1.11900,24.4,8.0,0.3279,0.40,-97.60,16.0,0.16
4,1.11930,1.11820,32.4,11.0,0.3395,0.60,-194.40,24.0,0.12
5,1.11860,1.11740,40.4,12.0,0.2970,0.92,-371.68,36.8,0.10
6,1.11800,1.11660,48.4,14.0,0.2893,1.36,-658.24,54.4,0.08
7,1.11730,1.11580,56.4,15.0,0.2660,1.96,-1105.44,78.4,0.07
8,1.11660,1.11500,64.4,16.0,0.2484,2.80,-1803.20,112.0,0.06
9,1.11590,1.11420,72.4,17.0,0.2348,3.96,-2867.04,158.4,0.06


In [45]:
# TODO: 封装方法计算最优参数

In [46]:
# 超参数: 

lots_factor = [2,3,4,5,8,11,15,21,29,30,30,30,30,30]
step_pips = 8
init_lots = 0.05
profit_per_lot_per_pip = 1

# 每组订单平仓目标位
target_pips = 2

# 回调20%触发 部分平仓 或 平盈利仓操作
retrace_act_pips_or_ratio = 0.2
# 部分平仓 || 平盈利仓 的 比例
retrace_act_lots_ratio = 0.4

In [47]:
# 被套之后，假设需要回调50%，